<a href="https://colab.research.google.com/github/sunshineluyao/ESG_TwoSources/blob/main/Sorting/Abnormal_Return_MSCI_2021_2_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Creat the Leader, Average, and Laggard Group

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df =pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Sorting/MSCI_2020_for_Sorting.csv')
df =df[['Ticker',	'Month',	'Size',	'Momentum', 'Industry_adjusted_BMR_Quarterly',	'Size_G',	'BMR_G',	'Mom_G','Monthly_Return','Porfolio_Return']]
df.head()

,Ticker,Month,Size,Momentum,Industry_adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Monthly_Return,Porfolio_Return
0,A,1,26461.710,11.63787,-0.124750,5,3,2,-3.223538,-2.903777
1,AAL,1,12280.780,-28.43626,-0.627498,4,1,1,-6.415624,-3.839508
2,AAON,1,2574.162,30.09222,-0.248233,2,1,4,6.132364,-0.441837
3,AAP,1,11092.590,-11.60880,-0.055731,4,3,1,-17.738520,-4.628674
4,AAPL,1,1304765.000,49.65282,-0.327109,5,1,5,5.400991,1.921124


In [ ]:
df['Abnormal_Return']=df['Monthly_Return']-df['Porfolio_Return']
df.head()

,Ticker,Month,Size,Momentum,Industry_adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Monthly_Return,Porfolio_Return,Abnormal_Return
0,A,1,26461.710,11.63787,-0.124750,5,3,2,-3.223538,-2.903777,-0.319761
1,AAL,1,12280.780,-28.43626,-0.627498,4,1,1,-6.415624,-3.839508,-2.576116
2,AAON,1,2574.162,30.09222,-0.248233,2,1,4,6.132364,-0.441837,6.574201
3,AAP,1,11092.590,-11.60880,-0.055731,4,3,1,-17.738520,-4.628674,-13.109846
4,AAPL,1,1304765.000,49.65282,-0.327109,5,1,5,5.400991,1.921124,3.479867


In [ ]:
ESG = pd.read_csv('https://raw.githubusercontent.com/sunshineluyao/ESG_TwoSources/main/Sorting/Separate_ESG_Score.csv')
ESG.head()

,Symbol,Name,Stock Exchange,RBICS Economy,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance
0,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9
1,AAL,"American Airlines Group, Inc.",NASDAQ,Industrials,B,2.3,3.4,3.1,3.7
2,AAN,Aaron's Company Inc,NYSE,Consumer Cyclicals,@NA,NaN,NaN,NaN,NaN
3,AAOI,"Applied Optoelectronics, Inc.",NASDAQ,Technology,@NA,NaN,NaN,NaN,NaN
4,AAON,"AAON, Inc.",NASDAQ,Industrials,A,7.1,5.5,6.5,5.5


In [ ]:
ESG.isna().sum()

Symbol                                                   0
Name                                                     0
Stock Exchange                                           0
RBICS Economy                                            0
MSCI ESG Rtg Overall Company Rating                      0
MSCI ESG Rtg Overall Industry-Adjusted Company Score    65
MSCI ESG Rtg Score Environmental                        65
MSCI ESG Rtg Score Social                               65
MSCI ESG Rtg Score Governance                           65
dtype: int64

In [ ]:
ESG=ESG.dropna()

In [ ]:
ESG['ESG_G']=pd.qcut(ESG['MSCI ESG Rtg Overall Industry-Adjusted Company Score'],5,labels=np.arange(1,6,1))
ESG['E_G']=pd.qcut(ESG['MSCI ESG Rtg Score Environmental'],5,labels=np.arange(1,6,1))
ESG['S_G']=pd.qcut(ESG['MSCI ESG Rtg Score Social'],5,labels=np.arange(1,6,1))
ESG['G_G']=pd.qcut(ESG['MSCI ESG Rtg Score Governance'],5,labels=np.arange(1,6,1))

In [ ]:
ESG['ESG_Group']=ESG['ESG_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['E_Group']=ESG['E_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['S_Group']=ESG['S_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')
ESG['G_Group']=ESG['G_G'].apply(lambda x: 'Laggard' if x==1 else 'Leader' if x==5 else 'Average')

In [ ]:
df=df.merge(ESG, left_on='Ticker', right_on='Symbol')
df.head()

,Ticker,Month,Size,Momentum,Industry_adjusted_BMR_Quarterly,Size_G,BMR_G,Mom_G,Monthly_Return,Porfolio_Return,Abnormal_Return,Symbol,Name,Stock Exchange,RBICS Economy,MSCI ESG Rtg Overall Company Rating,MSCI ESG Rtg Overall Industry-Adjusted Company Score,MSCI ESG Rtg Score Environmental,MSCI ESG Rtg Score Social,MSCI ESG Rtg Score Governance,ESG_G,E_G,S_G,G_G,ESG_Group,E_Group,S_Group,G_Group
0,A,1,26461.71,11.637870,-0.124750,5,3,2,-3.223538,-2.903777,-0.319761,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
1,A,2,25608.71,26.460120,-0.124750,5,3,2,-6.649709,-7.847952,1.198243,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
2,A,3,23950.89,8.574427,-0.124750,5,3,2,-7.071489,-12.705259,5.633770,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
3,A,4,22502.34,-5.132937,-0.092962,5,3,1,11.230420,8.530156,2.700264,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average
4,A,5,23737.85,-12.187340,-0.092962,5,3,2,18.771050,8.965679,9.805371,A,"Agilent Technologies, Inc.",NYSE,Healthcare,AA,8.2,10.0,6.8,5.9,5,5,5,4,Leader,Leader,Leader,Average


# ESG: Visualizing Abnormal Return Monthly

In [ ]:
import plotly.express as px
fig = px.box(df, x="ESG_Group", y="Abnormal_Return",color='ESG_Group',animation_frame='Month',animation_group='Symbol')
fig.show()

In [ ]:
ESG_summary = df.groupby(by=['Month','ESG_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
ESG_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
ESG_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
ESG_summary.head()

,Month,ESG_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,-0.160616,8.336157,837,28.930952,0.288140
1,1,Laggard,0.559413,8.648129,285,16.881943,0.512271
2,1,Leader,0.074271,7.309727,273,16.522712,0.442405
3,2,Average,-0.235725,8.541001,837,28.930952,0.295220
4,2,Laggard,0.149127,9.573440,285,16.881943,0.567082


In [ ]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

In [ ]:
ESG_summary = ESG_summary[ESG_summary['ESG_Group'].isin(['Leader', 'Laggard'])]

In [ ]:
import plotly.express as px
fig = px.line(ESG_summary, x="Month", y="mean", color="ESG_Group", error_y="mean_std")
fig.show()

### E result

In [ ]:
E_summary = df.groupby(by=['Month','E_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
E_summary['sqrt_count']=ESG_summary['count'].apply(lambda x: np.sqrt(x))
E_summary['mean_std']=ESG_summary['std']/ESG_summary['sqrt_count']
E_summary.head()

,Month,E_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.562446,8.384935,846,NaN,NaN
1,1,Laggard,-2.547513,8.315611,280,16.881943,0.512271
2,1,Leader,1.051106,6.937067,269,16.522712,0.442405
3,2,Average,-0.068920,8.948526,846,NaN,NaN
4,2,Laggard,-1.350393,7.716570,280,16.881943,0.567082


In [ ]:
E_summary = E_summary[E_summary['E_Group'].isin(['Leader', 'Laggard'])]

In [ ]:
import plotly.express as px
fig = px.line(E_summary, x="Month", y="mean", color="E_Group", error_y="mean_std")
fig.show()

### S result

In [ ]:
S_summary = df.groupby(by=['Month','S_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
S_summary['sqrt_count']=S_summary['count'].apply(lambda x: np.sqrt(x))
S_summary['mean_std']=S_summary['std']/S_summary['sqrt_count']
S_summary.head()

,Month,S_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.200407,8.318087,845,29.068884,0.286151
1,1,Laggard,-0.066149,7.976692,304,17.435596,0.457495
2,1,Leader,-0.422607,8.138069,246,15.684387,0.518864
3,2,Average,-0.170265,8.622233,845,29.068884,0.296614
4,2,Laggard,0.101858,8.614452,304,17.435596,0.494073


In [ ]:
S_summary = S_summary[S_summary['S_Group'].isin(['Leader', 'Laggard'])]

In [ ]:
import plotly.express as px
fig = px.line(S_summary, x="Month", y="mean", color="S_Group", error_y="mean_std")
fig.show()

### G result

In [ ]:
G_summary = df.groupby(by=['Month','G_Group'],as_index=False)['Abnormal_Return'].agg(['mean','std','count']).reset_index()
G_summary['sqrt_count']=G_summary['count'].apply(lambda x: np.sqrt(x))
G_summary['mean_std']=G_summary['std']/G_summary['sqrt_count']
G_summary.head()

,Month,G_Group,mean,std,count,sqrt_count,mean_std
0,1,Average,0.258664,7.808175,823,28.687977,0.272176
1,1,Laggard,0.110504,8.675947,306,17.492856,0.495971
2,1,Leader,-0.757225,8.837410,266,16.309506,0.541856
3,2,Average,-0.061412,8.440481,823,28.687977,0.294217
4,2,Laggard,-0.164686,9.812369,306,17.492856,0.560936


In [ ]:
G_summary = G_summary[G_summary['G_Group'].isin(['Leader', 'Laggard'])]

In [ ]:
import plotly.express as px
fig = px.line(G_summary, x="Month", y="mean", color="G_Group", error_y="mean_std")
fig.show()

# Step 3: Conduct Two-group t-test for each month

In [ ]:
pip install pingouin

     |████████████████████████████████| 225kB 5.4MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.9-cp36-none-any.whl size=223375 sha256=e835e56fe82822b64314bde202a9923d1c7f4e357e6c39d15c91c2a2f0c44569
  Stored in directory: /root/.cache/pip/wheels/6c/d7/26/a23b9ed74061adc5cf8f5e016bbe2902a8ebf63b6471047dbc
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=0b7467247ad6c2633084c5f4e11ced8264e572be26cdd2ec47e46a8300a7ad60
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=0d95b7ff960b8755152f81dcc16796a910f0376b22d86cb1db0cc8ca05a8b273
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


In [ ]:
import pingouin as pg


Two-Sample T-Test

https://pingouin-stats.org/generated/pingouin.ttest.html#pingouin.ttest

### Test Monthly the Leaders' Abornormal Return >0?

In [ ]:
Leader = df[(df['Month']==1) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,0.167881,272,two-sided,0.866802,"[-0.8, 0.95]",0.010161,0.069,0.053212


In [ ]:
df_ = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader =  df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  t_test = pg.ttest(Leader, 0)
  t_test['Month']=i
  df_=df_.append(t_test)

In [ ]:
df_

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,0.167881,272,two-sided,0.866802,"[-0.8, 0.95]",0.010161,0.069,0.053212,1
T-test,0.608057,272,two-sided,0.543657,"[-0.64, 1.21]",0.036801,0.081,0.093006,2
T-test,1.262381,272,two-sided,0.207893,"[-0.61, 2.8]",0.076403,0.149,0.241973,3
T-test,-0.492051,272,two-sided,0.623080,"[-2.15, 1.29]",0.029780,0.076,0.077966,4
T-test,0.544407,272,two-sided,0.586607,"[-0.84, 1.47]",0.032949,0.079,0.084339,5
T-test,1.732882,272,two-sided,0.084250,"[-0.11, 1.67]",0.104879,0.297,0.407916,6
T-test,1.718872,272,two-sided,0.086775,"[-0.15, 2.27]",0.104031,0.29,0.402512,7
T-test,1.066399,272,two-sided,0.287189,"[-0.49, 1.65]",0.064541,0.119,0.186025,8
T-test,1.529807,272,two-sided,0.127227,"[-0.22, 1.76]",0.092588,0.215,0.331826,9
T-test,0.217627,272,two-sided,0.827883,"[-0.88, 1.1]",0.013171,0.069,0.055405,10


In [ ]:
df_2 = pd.DataFrame()
Month_List =df.Month.unique().tolist()
for i in Month_List:
  Leader = df[(df['Month']==i) & (df['ESG_Group']=='Leader')]['Abnormal_Return']
  Laggard = df[(df['Month']==i) & (df['ESG_Group']=='Laggard')]['Abnormal_Return']
  t_test = pg.ttest(Leader, Laggard,paired=False)
  t_test['Month']=i
  df_2=df_2.append(t_test)

In [ ]:
df_2

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power,Month
T-test,-0.716750,547.576034,two-sided,0.473834,"[-1.81, 0.84]",0.060482,0.121,0.109956,1
T-test,0.184709,540.979265,two-sided,0.853526,"[-1.31, 1.58]",0.015572,0.096,0.053869,2
T-test,1.942290,537.773463,two-sided,0.052624,"[-0.03, 5.37]",0.163686,0.59,0.487903,3
T-test,-0.144960,528.845241,two-sided,0.884798,"[-3.01, 2.6]",0.012205,0.095,0.052375,4
T-test,1.004113,553.624700,two-sided,0.315763,"[-0.83, 2.58]",0.084836,0.154,0.170085,5
T-test,1.690606,537.410587,two-sided,0.091492,"[-0.2, 2.62]",0.142470,0.379,0.389665,6
T-test,1.714283,553.323772,two-sided,0.087037,"[-0.21, 3.13]",0.145259,0.394,0.402317,7
T-test,1.880299,551.900321,two-sided,0.060594,"[-0.06, 2.86]",0.159384,0.526,0.467716,8
T-test,2.355682,548.672784,two-sided,0.018840,"[0.27, 2.93]",0.199806,1.394,0.653708,9
T-test,1.725597,532.777207,two-sided,0.085000,"[-0.19, 3.0]",0.145348,0.401,0.402720,10


# Overall test

In [ ]:
Leader =df[df['ESG_Group']=='Leader']['Abnormal_Return']
Laggard=df[df['ESG_Group']=='Laggard']['Abnormal_Return']

In [ ]:
t_test = pg.ttest(Leader, 0)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.26731,3275,two-sided,0.023436,"[0.05, 0.73]",0.039613,0.256,0.620469


In [ ]:
t_test = pg.ttest(Leader, Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,3.406333,6554.397072,two-sided,0.000662,"[0.39, 1.43]",0.082938,8.955,0.923945


In [ ]:
E_Leader =df[df['E_Group']=='Leader']['Abnormal_Return']
E_Laggard=df[df['E_Group']=='Laggard']['Abnormal_Return']

In [ ]:
t_test = pg.ttest(E_Leader, E_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,2.522502,6405.576009,two-sided,0.011676,"[0.15, 1.22]",0.061911,0.663,0.709435


In [ ]:
S_Leader =df[df['S_Group']=='Leader']['Abnormal_Return']
S_Laggard=df[df['S_Group']=='Laggard']['Abnormal_Return']

In [ ]:
t_test = pg.ttest(S_Leader, S_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,1.848526,6248.723451,two-sided,0.064573,"[-0.03, 1.06]",0.045881,0.153,0.457493


In [ ]:
G_Leader =df[df['G_Group']=='Leader']['Abnormal_Return']
G_Laggard=df[df['G_Group']=='Laggard']['Abnormal_Return']

In [ ]:
t_test = pg.ttest(G_Leader, G_Laggard, paired=False)
t_test

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,3.01841,6728.686439,two-sided,0.002551,"[0.3, 1.4]",0.073047,2.562,0.855004


In [ ]:
df.to_csv("ESG_MSCI_FINAL.csv")